In [ ]:
!pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os   
import csv
import pandas as pd
import numpy as np
import math
import keras
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from imblearn.over_sampling import SMOTE 
from google.colab import drive
import librosa
import pickle


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

df = pd.read_csv('/content/drive/MyDrive/SE4CSAI/warblrb10k_public_metadata_2018.csv')
df.drop('datasetid', inplace=True, axis=1)
labels = dict(df.values)
labels

FileNotFoundError: ignored

In [ ]:
TARGET_SR = 22050

In [ ]:
def wav_loader(filename):
    signal, sr = librosa.load(filename, sr=22050, mono=True)
    return signal

def preprocess(filename, label):
    wav = wav_loader(filename)
    wav = wav[:TARGET_SR*3]
   
    zero_padding = tf.zeros([TARGET_SR*3]-tf.shape(wav), dtype=tf.float32)
    
    wav = tf.concat([zero_padding, wav],0)
    wav = wav.numpy()
    
    mel = librosa.feature.melspectrogram(wav, sr=TARGET_SR, n_fft=2048,hop_length=512,n_mels=128,fmin=20,fmax=8300)
    mel = librosa.power_to_db(mel,top_db=80)
    return mel, label

def preprocess_for_pred(filename):
  wav = wav_loader(filename)
  wav = wav[:22050*3]
  
  zero_padding = tf.zeros([22050*3]-tf.shape(wav), dtype=tf.float32)
  
  wav = tf.concat([zero_padding, wav],0)
  wav = wav.numpy()
  
  mel = librosa.feature.melspectrogram(wav, sr=TARGET_SR, n_fft=2048,hop_length=512,n_mels=128,fmin=20,fmax=8300)
  mel = librosa.power_to_db(mel,top_db=80)
  mel = mel.reshape(1,128,130)
  mel = tf.convert_to_tensor(mel)

  return mel

In [ ]:
#filling dataframe with arrays of wav files
audio_dataframe = pd.DataFrame(columns=('filename', 'label', 'tensor_wave', 'length'))
directory = '/content/drive/MyDrive/SE4CSAI/bird_audio'
i = 0

 
# iterate over files in
# that directory

for filename in os.listdir(directory):
    
    
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        #loading audio files
        
        
        
        #Remove .wav
        filename = filename.replace('.wav', '')
        #Removing doubles
        if '(1)' in filename:
            continue
        tensor_wave, __ = preprocess(f, labels[filename])
        length = len(tensor_wave)
        
            
        #filling dataframe with filename, binary label and tensor
        audio_dataframe.loc[i] = [filename,float(labels[filename]),tensor_wave,length]
        i += 1
       
       
audio_dataframe

FileNotFoundError: ignored

In [ ]:
with open('/content/drive/MyDrive/SE4CSAI/audio_labels.pkl', 'rb') as f:
  labels = pickle.load(f)



In [ ]:
load_arrays = np.load('/content/drive/MyDrive/SE4CSAI/bird_audio_arrays.npy')
print(load_arrays)

In [ ]:
#filling tensorflow dataset with tensors and labels

tensor_audio = list(load_arrays)
tensor_labels = labels
tensor_data_set = tf.data.Dataset.zip((tf.data.Dataset.from_tensor_slices(tensor_audio), 
                                       tf.data.Dataset.from_tensor_slices(tensor_labels)))
samples = tensor_data_set.as_numpy_iterator().next()
print(samples)

In [ ]:
i = 0
for element in tensor_data_set:
  print(element[0]) #returns tensors
  i += 1
  if i == 10:
    break

In [ ]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
      

In [ ]:
reshaped_input = []
reshaped_label = []
for element in tensor_data_set:
  label = element[1]
  element = element[0].reshape(1,128,130)
  label = label.reshape(-1,1)

  
  reshaped_label.append(label)
  reshaped_input.append(element)


In [ ]:
tensor_data_set = tf.data.Dataset.zip((tf.data.Dataset.from_tensor_slices(reshaped_input), 
                                       tf.data.Dataset.from_tensor_slices(reshaped_label)))



In [ ]:
data = tensor_data_set
data = data.cache()
train_data = data.take(7000) #takes 800 batches of 8 
validation_data = data.take(1000) #takes 200 batches of 8

samples = train_data.as_numpy_iterator().next()
print(samples[0].shape)


In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler, SMOTE
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D

model=Sequential()
#adding convolution layer
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(128,30)))
#adding pooling layer
model.add(MaxPool2D(2,2))
#adding fully connected layer
model.add(Flatten())
model.add(Dense(100, activation='relu'))
#adding output layer
model.add(Dense(10,activation='softmax'))
#compiling the model
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
#fitting the model
model.fit(train_data ,epochs=10)

In [ ]:

model = keras.Sequential()

# Convolutional layer and maxpool layer 1
model.add(keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(128,130,1), padding='same'))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 2
model.add(keras.layers.Conv2D(64,(3,3),activation='relu', padding='same'))
model.add(keras.layers.MaxPool2D(2,2))

#model.add(keras.layers.Conv2D(128,(3,3),activation='relu', padding='same'))
#model.add(keras.layers.MaxPool2D(2,2))


# This layer flattens the resulting image array to 1D array
model.add(keras.layers.Flatten())

# Hidden layer with 512 neurons and Rectified Linear Unit activation function 

model.add(keras.layers.Dense(256,activation='relu'))

model.add(keras.layers.Dense(1,activation='sigmoid'))

model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_36 (Conv2D)          (None, 128, 130, 32)      320       
                                                                 
 max_pooling2d_36 (MaxPoolin  (None, 64, 65, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_37 (Conv2D)          (None, 64, 65, 64)        18496     
                                                                 
 max_pooling2d_37 (MaxPoolin  (None, 32, 32, 64)       0         
 g2D)                                                            
                                                                 
 flatten_15 (Flatten)        (None, 65536)             0         
                                                                 
 dense_33 (Dense)            (None, 256)             

In [ ]:

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
pos = (labels.count(1.0))
neg = (labels.count(0.0))

weight_for_0 = (1 / neg) * (len(labels) / 2.0)
weight_for_1 = (1 / pos) * (len(labels) / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

In [ ]:
initial_bias = np.log([pos/neg])

In [ ]:

with tf.device('/GPU:0'):
  history = model.fit(train_data,
                      steps_per_epoch = 7000,
                      epochs = 8,
                      validation_data = validation_data,
                      batch_size = 8,
                      class_weight = class_weight,
                      )


Epoch 1/8
7000/7000 [==============================] - 47s 7ms/step - loss: 1.4863 - accuracy: 0.6530 - val_loss: 0.6339 - val_accuracy: 0.7060
Epoch 2/8
7000/7000 [==============================] - 49s 7ms/step - loss: 0.6977 - accuracy: 0.7126 - val_loss: 0.5878 - val_accuracy: 0.7160
Epoch 3/8
7000/7000 [==============================] - 46s 7ms/step - loss: 0.5571 - accuracy: 0.7819 - val_loss: 0.4315 - val_accuracy: 0.8030
Epoch 4/8
7000/7000 [==============================] - 46s 7ms/step - loss: 0.4159 - accuracy: 0.8287 - val_loss: 0.5101 - val_accuracy: 0.7260
Epoch 5/8
7000/7000 [==============================] - 46s 7ms/step - loss: 0.3114 - accuracy: 0.8947 - val_loss: 0.2341 - val_accuracy: 0.9110
Epoch 6/8
7000/7000 [==============================] - 46s 7ms/step - loss: 0.2398 - accuracy: 0.9219 - val_loss: 0.1790 - val_accuracy: 0.9360
Epoch 7/8
7000/7000 [==============================] - 46s 7ms/step - loss: 0.1972 - accuracy: 0.9390 - val_loss: 0.2640 - val_accuracy:

In [ ]:
model.save('/content/drive/MyDrive/SE4CSAI/binary_class_model_jurre')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/SE4CSAI/binary_class_model_jurre')

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

def preprocess_for_pred(filename):
  wav, _ = librosa.load(filename, sr=22050, mono=True)
  wav = wav[:22050*3]
  
  zero_padding = tf.zeros([22050*3]-tf.shape(wav), dtype=tf.float32)
  
  wav = tf.concat([zero_padding, wav],0)
  wav = wav.numpy()
  
  mel = librosa.feature.melspectrogram(wav, sr=TARGET_SR, n_fft=2048,hop_length=512,n_mels=128,fmin=20,fmax=8300)
  mel = librosa.power_to_db(mel,top_db=80)
  mel = mel.reshape(1,128,130)
  mel = tf.convert_to_tensor(mel)

  return mel

In [ ]:
def make_prediction(audiofile):
  data = preprocess_for_pred(audiofile)
  prediction = model.predict(data)
  return prediction